In [1]:
# test for genes related to chicken IntX evolution
suppressPackageStartupMessages({
   library(Seurat)
    library(clusterProfiler)
    library(enrichplot)
    library(ggplot2)
    library(org.Ggallus.eg.db) 
})

Warning message:
“package ‘Biobase’ was built under R version 4.3.3”
Warning message:
“package ‘IRanges’ was built under R version 4.3.3”
Warning message:
“package ‘S4Vectors’ was built under R version 4.3.3”


In [2]:
Ggal_bg <- read.table('../2.ohnolog_significance/2.bg/chicken.bg')$V1
Ggal_TFs <- read.table('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/01.data/07.species_signals/6.TF_vs_species_signals/TFs/Ggal.predicted_TFs.txt', header = T)
Ggal_TFs <- Ggal_TFs[Ggal_TFs$prediction == 'True', 1]
Ggal_ID <- read.delim('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/13.case_analysis/1.cerebellum/2.ohnolog_significance/2.bg/chicken.info')

In [3]:
gain_loss <- read.delim('/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/13.case_analysis/1.cerebellum/3.genes_drive_evolution/Ggal.gain_loss.txt', header = F)

In [4]:
tmp1 <- gain_loss %>% filter(V2 == 'gain' & V1 == 'IntX.B')
tmp2 <- gain_loss %>% filter(V2 == 'gain' & V1 == 'IntX.A')
tmp <- rbind(tmp1, tmp2)

In [5]:
tmp$Ensembl <- vapply(tmp$V3, FUN = function(x){
    if (x %in% Ggal_ID$Gene.name){
        Ggal_ID[Ggal_ID$Gene.name == x, 1][1]
    } else {
        x
    }
}, FUN.VALUE = character(1))
tmp$TF <- tmp$Ensembl %in% Ggal_TFs

In [6]:
#ego2 <- enrichGO(gene         = unique(tmp$Ensembl),
#                 OrgDb         = org.Ggallus.eg.db,
#                 keyType = "GID",
#                 pAdjustMethod = "BH",
#                 pvalueCutoff = 0.05,
#                 universe = Ggal_bg,
#                 ont  = "BP")
#ego2 <- simplify(ego2, cutoff=0.7, by="p.adjust", select_fun=min)

In [7]:
Ggal <- readRDS('../1.update_atlas/GSM4873767_chicken_data.coding_genes.rds')
Ggal@meta.data$cell_type <- factor(Ggal@meta.data$cell_type, levels = c('Med.A', 'IntP.A', 'IntP.B', 'MedL.B', 'Med.B', 'IntX.B', 'IntX.A', 'MedL.A'))
Idents(Ggal) <- 'cell_type'

In [8]:
pdf('results/Ggal.IntX.TFs.pdf', width = 8, height = 8)
DotPlot(Ggal, features = rev(tmp[tmp$TF, 'V3'])) + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
dev.off()

pdf 
  2